We currently have the following for our `Sort` class,

In [ ]:
public class Sort {

    public static void sort(String[] x){
        sort(x, 0); // Call the helper sort method
    }

    private static void sort(String[] x, int start) {
        int smallestIndex = findSmallest(x); // Find the smallest index during
        swap(x, start, smallestIndex); // Swap the item at index[start] with the item at index [smallestIndex]
        sort(x, start + 1); // Recursive call sort for the next starting point
    }

    public static int findSmallest(String[] x) {
        int smallestIndex = 0;
        for (int i = 0; i < x.length; i+= 1) {
            int comparison = x[i].compareTo(x[smallestIndex]); // if x[i] is less than x[smallestIndex], it will return a negative number
            if (comparison < 0) {
                smallestIndex = i;
            }
        }
        return smallestIndex;
    }

    public static void swap(String[] x, int a, int b) {
        String temp = x[a];
        x[a] = x[b];
        x[b] = temp;
    }
}

If we try to run the `testSort()` test,

In [ ]:
OpenJDK 64-Bit Server VM warning: Sharing is only supported for boot loader classes because bootstrap classpath has been appended
Exception in thread "main" java.lang.ArrayIndexOutOfBoundsException: Index 4 out of bounds for length 4
	at Sort.swap(Sort.java:37)
	at Sort.sort(Sort.java:9)
	at Sort.sort(Sort.java:10)
	at Sort.sort(Sort.java:10)
	at Sort.sort(Sort.java:10)
	at Sort.sort(Sort.java:10)
	at Sort.sort(Sort.java:4)
	at TestSort.testSort(TestSort.java:8)
	at TestSort.main(TestSort.java:38)

An `IndexOutOfBoundsException` error! To analyze this issue, we set a breakpoint in within the `swap` method and set the condition when `a == 4`

In [ ]:
public static void swap(String[] x, int a, int b) {
        String temp = x[a]; // SET A BREAKPOINT HERE
        x[a] = x[b];
        x[b] = temp;
    }

<img src = 'break.png' width = 400/>

This time, if we debug the code and look at Java Visualizer, notice that `start` is `4`. This is not supposed to happen since the maximum index possible is 3! We need a base case!

By utilizing testing and debugging with Java Visualizer, we can easily notice where things go wrong and realize that we need to implement a base case. Let's implement the base case.

In [ ]:
private static void sort(String[] x, int start) {
        if (start == x.length) { // If start gets too big, just return nothing to stop the recursive call
            return;
        }
        int smallestIndex = findSmallest(x); // Find the smallest index during
        swap(x, start, smallestIndex); // Swap the item at index[start] with the item at index [smallestIndex]
        sort(x, start + 1); // Recursive call sort for the next starting point
    }

Now that a base case is implemented, run the `testSort()` once again!

In [ ]:
Exception in thread "main" arrays first differed at element [0]; expected:<[an]> but was:<[have]>
	at org.junit.internal.ComparisonCriteria.arrayEquals(ComparisonCriteria.java:55)
	at org.junit.Assert.internalArrayEquals(Assert.java:532)
	at org.junit.Assert.assertArrayEquals(Assert.java:283)
	at org.junit.Assert.assertArrayEquals(Assert.java:298)
	at TestSort.testSort(TestSort.java:10)
	at TestSort.main(TestSort.java:38)
Caused by: org.junit.ComparisonFailure: expected:<[an]> but was:<[have]>
	at org.junit.Assert.assertEquals(Assert.java:115)
	at org.junit.Assert.assertEquals(Assert.java:144)
	at org.junit.internal.ExactComparisonCriteria.assertElementsEqual(ExactComparisonCriteria.java:8)
	at org.junit.internal.ComparisonCriteria.arrayEquals(ComparisonCriteria.java:53)
	... 5 more

We still failed the test! This time, it seems that there's a problem with the sorting process somehow that the first element of the resulting array is `"have"` instead of `"an"`! 

There could be a problem with the elements swapping. Let's set a break point at the `swap` process and see if we find anything.

In [ ]:
private static void sort(String[] x, int start) {
        if (start == x.length) {
            return;
        }
        int smallestIndex = findSmallest(x);
        swap(x, start, smallestIndex); // SET BREAKPOINT HERE
        sort(x, start + 1);
    }

If we look at the Java Visualizer and keep going until we find the error:
* At first, Java successfully swap the first round
    * "an" and "i" is swapped
* At the second round, when Java tried to find the smallest index via `findSmallest`,
    * `findSmallest` happened to also include the element at the front, "an", which is not supposed to happen!

<img src = 'problem.png' width = 700/>

We have a design flaw in the `findSmallest` method! It shouldn't return the smallest String in the whole array `x`, but instead it should return the array starting from a `start`ing point.

In [ ]:
public static int findSmallest(String[] x, int start) {
        int smallestIndex = start; // starting from start
        for (int i = start; i < x.length; i+= 1) { // iterate starting from start
            int comparison = x[i].compareTo(x[smallestIndex]); 
            if (comparison < 0) {
                smallestIndex = i;
            }
        }
        return smallestIndex;
    }

Since we modified the `findSmallest` method, we might become unsure if the method if still works fine. In this case, we can test it again using `testFindSmallest`! 

However, we'll need to modify the `testFindSmallest` as well since we modified the signature of `findSmallest` method.

In [ ]:
public static void testFindSmallest() {
    String[] input = {"i", "have", "an", "egg"};
    int expected = 2;

    int actual = Sort.findSmallest(input, 0); // ADD AN ARGUMENT OF STARTING POINT 0
    org.junit.Assert.assertEquals(expected, actual);

    String[] input2 = {"there", "are", "many", "pigs"};
    int expected2 = 1;

    int actual2 = Sort.findSmallest(input2, 0); // ADD AN ARGUMENT OF STARTING POINT 0
    org.junit.Assert.assertEquals(expected2, actual2);
}


In [ ]:
    public static void main(String[] args) {
//        testSort();
        testFindSmallest();
//        testSwap();
    }


Now that we pass the `testFindSmallest` test, test the `tesstSort()` method as a whole! Remember that also need to add a `start` when calling `findSmallest` in our `Sort` method!

In [ ]:
private static void sort(String[] x, int start) {
    if (start == x.length) {
        return;
    }
    int smallestIndex = findSmallest(x, start); // ADD A STARTING POINT
    swap(x, start, smallestIndex); 
    sort(x, start + 1); 
}


We should pass all the tests smoothly this time!

The main idea is that testing helps us with isolating one activity at a time. When we found out that there is a problem with the swapping, we can focus on the `testFindSmallest` tests rather than testing `testSort` as a whole.